In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os
import boto3

In [ ]:
data_folder = "../../data"
test_file_path = os.path.join(data_folder, "merged_instructions.csv")
df = pd.read_csv(test_file_path)
df['english_instruction'].head()

In [ ]:

result_data = []
client = boto3.client("bedrock-runtime", region_name="us-west-2")

for index, row in df.iterrows():
    message = row['instruction']
    model_id = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'

    prompt = f"You are a helpful assistant that generates code snippets based on user instructions." \
                "The method signature should be according to the given in the example part.\n" \
                "Use the following guidlines in generation: \n" \
                " - Provide the code in the response only\n" \
                " - Do not output any other texts or test cases results.\n" \
                " - Include the necessary import statements on the top even if they are usually not needed like - re, typing, itertools, Split etc. \n" \
                " - Do not include any comments in the code\n" \
                " - Do not forget to consider the edge cases\n" \
                " - Avoid recursion if possible\n" \
                " - Lastly, verify the code has no compilations error and contains the necessary imports.\n"
    
    # Set up messages and system message
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "text": message
                }
            ]
        }
    ]

    system = [
        {
            "text": prompt
        }
    ]

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId=model_id,
            messages=messages,
            system=system,
            inferenceConfig={
                "maxTokens": 1024, 
                "temperature": 0.5
            },
        )

        # Extract and print the response text.
        flag = False
        for content_block in response["output"]["message"]["content"]:
            if content_block.get("text"):
                code_response = content_block.get("text")
                result_data.append({
                    "id": row['id'],
                    "response": code_response
                })
                flag = True

        if not flag:
            result_data.append({
                    "id": row['id'],
                    "response": ""
                })

    except (Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")

    print("Completed:", row['id'])
    
result_df = pd.DataFrame(result_data)

In [ ]:
# Create a json from the result_df
result_json = result_df.to_json(orient="records", indent=4)
# save json in the data_folder os.path.join(data_folder, "submission.json")
with open("claude_3.5_haiku_submission.json", "w") as json_file:
    json_file.write(result_json)